### Linear and Multilinear Functions

In [1]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [2]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(1.44 𝐞₁ + -0.85 𝐞₂ + 2.77 𝐞₃ + -1.11 𝐞₄,
 1.44 𝐞₁ + -0.85 𝐞₂ + 2.77 𝐞₃ + -1.11 𝐞₄)

In [3]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(3.79 𝐞₁ + 3.93 𝐞₂ + 1.39 𝐞₃ + -3.88 𝐞₄,
 3.79 𝐞₁ + 3.93 𝐞₂ + 1.39 𝐞₃ + -3.88 𝐞₄)

In [4]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [5]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(2.58 𝐞₁ + 1.78 𝐞₂ + 2.61 𝐞₃ + -1.47 𝐞₄,
 2.58 𝐞₁ + 1.78 𝐞₂ + 2.61 𝐞₃ + -1.47 𝐞₄)

In [6]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.0888 + 0.769 𝐞₁ + -1.63 𝐞₂ + 2.81 𝐞₃ + -0.943 𝐞₄ + 2.36 𝐞₁₂ + -10.1 𝐞₁₃ + 6.65 𝐞₁₄ + 12.7 𝐞₂₃ + -11.2 𝐞₂₄ + 12.0 𝐞₃₄,
 0.0888 + 0.769 𝐞₁ + -1.63 𝐞₂ + 2.81 𝐞₃ + -0.943 𝐞₄ + 2.36 𝐞₁₂ + -10.1 𝐞₁₃ + 6.65 𝐞₁₄ + 12.7 𝐞₂₃ + -11.2 𝐞₂₄ + 12.0 𝐞₃₄)

In [7]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(10.0 𝐞₁₂₃ + -1.67 𝐞₁₂₄ + -0.0235 𝐞₁₃₄ + -4.61 𝐞₂₃₄,
 10.0 𝐞₁₂₃ + -1.67 𝐞₁₂₄ + -0.0235 𝐞₁₃₄ + -4.61 𝐞₂₃₄)

In [8]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.696 + 1.48 𝐞₁ + 2.85 𝐞₂ + 3.53 𝐞₃ + -2.91 𝐞₄ + 7.35 𝐞₁₂ + -0.694 𝐞₁₃ + 1.48 𝐞₁₄ + -4.79 𝐞₂₃ + 3.4 𝐞₂₄ + 1.29 𝐞₃₄ + 1.47e+02 𝐞₁₂₃ + -99.5 𝐞₁₂₄ + 31.7 𝐞₁₃₄ + 49.3 𝐞₂₃₄ + 2.27e+02 𝐞₁₂₃₄,
 0.696 + 1.48 𝐞₁ + 2.85 𝐞₂ + 3.53 𝐞₃ + -2.91 𝐞₄ + 7.35 𝐞₁₂ + -0.694 𝐞₁₃ + 1.48 𝐞₁₄ + -4.79 𝐞₂₃ + 3.4 𝐞₂₄ + 1.29 𝐞₃₄ + 1.47e+02 𝐞₁₂₃ + -99.5 𝐞₁₂₄ + 31.7 𝐞₁₃₄ + 49.3 𝐞₂₃₄ + 2.27e+02 𝐞₁₂₃₄)

In [9]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.0888 + 1.46 𝐞₁ + 3.38 𝐞₂ + 4.54 𝐞₃ + -2.4 𝐞₄ + -7.19 𝐞₁₂ + 9.54 𝐞₁₃ + -9.28 𝐞₁₄ + 3.43 𝐞₂₃ + 2.42 𝐞₂₄ + -6.04 𝐞₃₄ + -1.29e+02 𝐞₁₂₃ + 89.0 𝐞₁₂₄ + -21.3 𝐞₁₃₄ + -46.7 𝐞₂₃₄ + 32.4 𝐞₁₂₃₄,
 0.0888 + 1.46 𝐞₁ + 3.38 𝐞₂ + 4.54 𝐞₃ + -2.4 𝐞₄ + -7.19 𝐞₁₂ + 9.54 𝐞₁₃ + -9.28 𝐞₁₄ + 3.43 𝐞₂₃ + 2.42 𝐞₂₄ + -6.04 𝐞₃₄ + -1.29e+02 𝐞₁₂₃ + 89.0 𝐞₁₂₄ + -21.3 𝐞₁₃₄ + -46.7 𝐞₂₃₄ + 32.4 𝐞₁₂₃₄)

In [10]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.0888 + 1.46 𝐞₁ + 3.38 𝐞₂ + 7.19 𝐞₁₂, 0.0888 + 1.46 𝐞₁ + 3.38 𝐞₂ + 7.19 𝐞₁₂)

In [11]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [12]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.0888 + -0.397 𝐞₁ + -0.465 𝐞₂ + 3.6 𝐞₃ + 1.8 𝐞₄ + 2.36 𝐞₁₂ + -4.39 𝐞₁₃ + -3.37 𝐞₁₄ + 16.3 𝐞₂₃ + 6.75 𝐞₂₄ + 10.7 𝐞₃₄,
 0.0888 + -0.397 𝐞₁ + -0.465 𝐞₂ + 3.6 𝐞₃ + 1.8 𝐞₄ + 2.36 𝐞₁₂ + -4.39 𝐞₁₃ + -3.37 𝐞₁₄ + 16.3 𝐞₂₃ + 6.75 𝐞₂₄ + 10.7 𝐞₃₄)

In [13]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.065 + -1.76 𝐞₁ + 1.6 𝐞₂ + 2.34 𝐞₃ + 1.24 𝐞₄ + -7.32 𝐞₁₂ + -6.92 𝐞₁₃ + -6.14 𝐞₁₄ + 5.65 𝐞₂₃ + 10.3 𝐞₂₄ + 17.4 𝐞₃₄ + 60.1 𝐞₁₂₃ + 0.193 𝐞₁₂₄ + -72.5 𝐞₁₃₄ + 1.08e+02 𝐞₂₃₄ + 46.1 𝐞₁₂₃₄,
 0.065 + -1.76 𝐞₁ + 1.6 𝐞₂ + 2.34 𝐞₃ + 1.24 𝐞₄ + -7.32 𝐞₁₂ + -6.92 𝐞₁₃ + -6.14 𝐞₁₄ + 5.65 𝐞₂₃ + 10.3 𝐞₂₄ + 17.4 𝐞₃₄ + 60.1 𝐞₁₂₃ + 0.193 𝐞₁₂₄ + -72.5 𝐞₁₃₄ + 1.08e+02 𝐞₂₃₄ + 46.1 𝐞₁₂₃₄)

In [14]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(-11.0 𝐞₁ + 4.7 𝐞₂ + 6.66 𝐞₃ + -15.0 𝐞₄,
 2.7 𝐞₁ + -2.0 𝐞₂ + -3.86 𝐞₃ + -2.23 𝐞₄,
 -11.0 𝐞₁ + 4.7 𝐞₂ + 6.66 𝐞₃ + -15.0 𝐞₄)

In [15]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(4.62 𝐞₁ + 7.94 𝐞₂ + -76.9 𝐞₃ + 53.6 𝐞₄,
 -15.5 𝐞₁ + -5.65 𝐞₂ + 0.378 𝐞₃ + -1.83 𝐞₄,
 4.62 𝐞₁ + 7.94 𝐞₂ + -76.9 𝐞₃ + 53.6 𝐞₄)

In [16]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

4.79 4.79
-9.3 -9.3
-40.3 -40.3


In [17]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(-11.0 𝐞₁ + 4.7 𝐞₂ + 6.66 𝐞₃ + -15.0 𝐞₄,
 -11.0 𝐞₁ + 4.7 𝐞₂ + 6.66 𝐞₃ + -15.0 𝐞₄)

In [18]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, 2.44e+02 𝐞₁₂₃₄, 2.44e+02 𝐞₁₂₃₄)

In [19]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(2.44e+02, 2.44e+02, 2.44e+02)

In [20]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-3.41 𝐞₁₃, -3.41 𝐞₁₃, -3.41 𝐞₁₃)

In [21]:
#1.17a
outermorphism(g, I, alg, h), Ip

(3.41 𝐞₂₃, 1 𝐞₂₃)

In [22]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(3.41 𝐞₁₂, 1 𝐞₁₂)

In [23]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

3.4101087755299084

In [24]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.621 + 0.542 𝐞₁ + 0.97 𝐞₂ + 0.415 𝐞₁₂,
 0.621 + 0.542 𝐞₁ + 0.97 𝐞₂ + 0.348 𝐞₃ + 0.98 𝐞₄ + 0.415 𝐞₁₂ + 0.634 𝐞₁₃ + 0.0406 𝐞₁₄ + 0.0932 𝐞₂₃ + 0.772 𝐞₂₄ + 0.116 𝐞₃₄ + 0.829 𝐞₁₂₃ + 0.655 𝐞₁₂₄ + 0.588 𝐞₁₃₄ + 0.531 𝐞₂₃₄ + 0.365 𝐞₁₂₃₄)

In [25]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.154 + 0.101 𝐞₂ + 0.441 𝐞₃ + 0.118 𝐞₂₃,
 0.154 + 0.37 𝐞₁ + 0.101 𝐞₂ + 0.441 𝐞₃ + 0.478 𝐞₄ + 0.644 𝐞₁₂ + 0.405 𝐞₁₃ + 0.665 𝐞₁₄ + 0.118 𝐞₂₃ + 0.295 𝐞₂₄ + 0.00518 𝐞₃₄ + 0.523 𝐞₁₂₃ + 0.15 𝐞₁₂₄ + 0.767 𝐞₁₃₄ + 0.787 𝐞₂₃₄ + 0.203 𝐞₁₂₃₄)

In [26]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(7.11 𝐞₁₂₃ + 6.58 𝐞₁₂₄ + 6.84 𝐞₁₃₄ + -0.206 𝐞₂₃₄,
 7.11 𝐞₁₂₃ + 6.58 𝐞₁₂₄ + 6.84 𝐞₁₃₄ + -0.206 𝐞₂₃₄,
 56.8 + -32.7 𝐞₁₂ + -26.7 𝐞₁₃ + -34.6 𝐞₁₄ + 2.39 𝐞₂₃ + -3.25 𝐞₂₄ + -30.0 𝐞₃₄,
 56.8 + -32.7 𝐞₁₂ + -26.7 𝐞₁₃ + -34.6 𝐞₁₄ + 2.39 𝐞₂₃ + -3.25 𝐞₂₄ + -30.0 𝐞₃₄)

In [27]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-5.35 + -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄,
 -5.35 + -7.78 𝐞₁ + 1.58 𝐞₂ + -0.4 𝐞₃ + 0.274 𝐞₄ + 1.42 𝐞₁₂ + 4.08 𝐞₁₃ + 0.638 𝐞₁₄ + -0.24 𝐞₂₃ + -1.24 𝐞₂₄ + -2.29 𝐞₃₄ + 2.78 𝐞₁₂₃ + -1.01 𝐞₁₂₄ + -2.13 𝐞₁₃₄ + 0.635 𝐞₂₃₄ + -0.752 𝐞₁₂₃₄)

In [28]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([0.0709 𝐞₁ + 6.41 𝐞₂ + 1.91 𝐞₃ + -2.07 𝐞₄,
  1.2 𝐞₁ + 2.92 𝐞₂ + 1.4 𝐞₃ + -0.92 𝐞₄,
  0.543 𝐞₁ + 5.29 𝐞₂ + 4.98 𝐞₃ + -2.58 𝐞₄,
  0.976 𝐞₁ + 2.21 𝐞₂ + 3.01 𝐞₃ + -0.799 𝐞₄],
 [0.0709 𝐞₁ + 6.41 𝐞₂ + 1.91 𝐞₃ + -2.07 𝐞₄,
  1.2 𝐞₁ + 2.92 𝐞₂ + 1.4 𝐞₃ + -0.92 𝐞₄,
  0.543 𝐞₁ + 5.29 𝐞₂ + 4.98 𝐞₃ + -2.58 𝐞₄,
  0.976 𝐞₁ + 2.21 𝐞₂ + 3.01 𝐞₃ + -0.799 𝐞₄])

In [29]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-5.35, -5.35, -5.35)

In [30]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(-9.15 + 2.27 𝐞₁₂ + 6.04 𝐞₁₃ + 14.9 𝐞₁₄ + -9.14 𝐞₂₃ + 19.9 𝐞₂₄ + 7.31 𝐞₃₄ + 10.5 𝐞₁₂₃₄,
 -9.15 + 2.27 𝐞₁₂ + 6.04 𝐞₁₃ + 14.9 𝐞₁₄ + -9.14 𝐞₂₃ + 19.9 𝐞₂₄ + 7.31 𝐞₃₄ + 10.5 𝐞₁₂₃₄)

In [31]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(77.38893423867682, 77.38893423867681, 77.3889342386768)

In [32]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(1.65e+02 + -70.9 𝐞₁₂ + -84.4 𝐞₁₃ + -94.0 𝐞₁₄ + 5.77 𝐞₂₃ + 13.3 𝐞₂₄ + -74.7 𝐞₃₄,
 1.65e+02 + -70.9 𝐞₁₂ + -84.4 𝐞₁₃ + -94.0 𝐞₁₄ + 5.77 𝐞₂₃ + 13.3 𝐞₂₄ + -74.7 𝐞₃₄)

In [33]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(1.01e+02, 101.04485146177167)

In [34]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(1.78 𝐞₁ + 34.3 𝐞₂ + 31.3 𝐞₃ + -23.9 𝐞₄,
 1.78 𝐞₁ + 34.3 𝐞₂ + 31.3 𝐞₃ + -23.9 𝐞₄,
 0.62 𝐞₁ + 0.578 𝐞₂ + 0.676 𝐞₃ + 0.251 𝐞₄,
 0.62 𝐞₁ + 0.578 𝐞₂ + 0.676 𝐞₃ + 0.251 𝐞₄)

In [35]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

-4.26e-14 𝐞₂ + -7.11e-14 𝐞₃

In [36]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([-4.59790929+1.30721359j, -4.59790929-1.30721359j,
         1.36219798+0.j        ,  2.48633971+0.j        ]),
 array([-4.59790929+1.30721359j, -4.59790929-1.30721359j,
         2.48633971+0.j        ,  1.36219798+0.j        ]))

In [37]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([-0.67471063, -0.67471063, -0.67471063, -0.67471063, -0.67471063,
       -0.67471063, -0.67471063, -0.67471063])

In [38]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(1.63 𝐞₁ + 3.25 𝐞₂ + 2.32 𝐞₃ + -1.02 𝐞₄,
 1.63 𝐞₁ + 3.25 𝐞₂ + 2.32 𝐞₃ + -1.02 𝐞₄)

In [39]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(-9.15399024488881, -9.153990244888805)

In [40]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(56.79226123828997, (56.79226123829+0j))

In [41]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(-129.36709722158122, (-129.36709722158128+0j))

In [42]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(77.4 𝐞₁₂₃₄, 77.4 𝐞₁₂₃₄, 77.4 𝐞₁₂₃₄)

In [43]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(-11.5 𝐞₁₂₃₄, -11.5 𝐞₁₂₃₄)

In [44]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(9.89 𝐞₁₂, 9.89 𝐞₁₂)

In [45]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-29.27120081877561, -29.27120081877561)

In [46]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-0.751 𝐞₁ + 0.497 𝐞₂, -0.751 𝐞₁ + 1.31 𝐞₂)

In [47]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(0.287 𝐞₃, 0.287 𝐞₃)

In [48]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(-0.287 𝐞₁₂, -0.287 𝐞₁₂)

In [49]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([2, 3], [(0, 1)]),
 array([-4.59790929+1.30721359j, -4.59790929-1.30721359j,
         1.36219798+0.j        ,  2.48633971+0.j        ]))

In [50]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[1.36219798 1.36219798 1.36219798 1.36219798] [ 2.51438877  9.17832951  2.10302016 -4.31495142]
[2.48633971 2.48633971 2.48633971 2.48633971] [  3.28325308   3.43894814   2.69757702 -17.37584088]
[22.84957718 22.84957718 22.84957718 22.84957718 22.84957718 22.84957718] [ 31.28439092  33.77430631 163.50836142   7.18314458  37.71336611
  40.01478835]


In [51]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[-1.71127696  8.29921842  7.99784201 -0.53748771] [1.36219798 1.36219798 1.36219798 1.36219798]
[ 9.09824903  6.79456235  7.31077928 -1.09672986] [2.48633971 2.48633971 2.48633971 2.48633971]
[12.01919848 17.15348839 -0.07380245 62.87769689 20.57210183 19.00558503] [22.84957718 22.84957718 22.84957718 22.84957718 22.84957718 22.84957718]


In [52]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, 1.11e-16, 1.0]

In [53]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(35.7 𝐞₁₂₃, 35.7 𝐞₁₂₃), (-1.71 𝐞₄, -1.71 𝐞₄)]

In [54]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[3.06472435 3.06472435 3.06472435] [3.58733806 2.80810499 2.77859329]
[] []
[11.63643486 11.63643486 11.63643486] [13.06340932 13.2223068   9.44082423]
[] []
[-1.70914638] [-1.70914638]
[] []


In [55]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[-0.16674087,  1.74703513,  1.810695  , -1.38057919],
        [ 1.74703513, -1.56805885,  2.8506107 ,  2.78032805],
        [ 1.810695  ,  2.8506107 , -1.90333478,  1.1635589 ],
        [-1.38057919,  2.78032805,  1.1635589 , -1.70914638]]),
 1.78 𝐞₁ + 2.8 𝐞₂ + 1.77 𝐞₃ + 1.11 𝐞₄,
 1.78 𝐞₁ + 2.8 𝐞₂ + 1.77 𝐞₃ + 1.11 𝐞₄)

In [56]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(-1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄,
 -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄)

In [57]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[ 5.55111512e-17,  6.55163325e-01, -9.48530295e-01,
          4.50963801e-01],
        [-6.55163325e-01,  1.11022302e-16,  8.11441158e-01,
          1.22274522e+00],
        [ 9.48530295e-01, -8.11441158e-01,  1.11022302e-16,
          1.68092334e+00],
        [-4.50963801e-01, -1.22274522e+00, -1.68092334e+00,
         -2.22044605e-16]]),
 -0.15 𝐞₁ + 0.45 𝐞₂ + 0.542 𝐞₃ + -2.12 𝐞₄,
 -0.15 𝐞₁ + 0.45 𝐞₂ + 0.542 𝐞₃ + -2.12 𝐞₄)

In [58]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(3.15, 3.15, 2.33e-10 𝐞₁₃ + 4.66e-10 𝐞₂₄ + -3.49e-10 𝐞₃₄)

In [59]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.78 𝐞₁ + 2.8 𝐞₂ + 1.77 𝐞₃ + 1.11 𝐞₄, 1.78 𝐞₁ + 2.8 𝐞₂ + 1.77 𝐞₃ + 1.11 𝐞₄)

In [60]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-9.48e+02 𝐞₁ + -1.62e-14 𝐞₂ + 3.38e-14 𝐞₃ + 1.58e-13 𝐞₄,
 1.23e-15 𝐞₁ + -71.9 𝐞₂ + 9.91e-15 𝐞₃ + -1.6e-14 𝐞₄,
 -1.41e-16 𝐞₁ + -5.44e-16 𝐞₂ + -3.95 𝐞₃ + 1.37e-15 𝐞₄,
 -1.67e-16 𝐞₁ + 2.22e-16 𝐞₂ + -3.47e-16 𝐞₃ + -0.998 𝐞₄]

In [61]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [62]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -5.35 + 2.22e-16 𝐞₁₄ + 8.88e-16 𝐞₂₃ + -8.88e-16 𝐞₂₄ + -4.44e-16 𝐞₃₄,
 -15.7 + 4.44e-16 𝐞₁₂ + -1.55e-15 𝐞₁₃ + 8.88e-16 𝐞₁₄ + -1.78e-15 𝐞₂₃ + 7.11e-15 𝐞₃₄ + -1.78e-15 𝐞₁₂₃₄,
 72.6 + 1.78e-15 𝐞₁₂ + -1.78e-15 𝐞₁₃ + -7.11e-15 𝐞₁₄ + -3.55e-15 𝐞₂₃ + -3.55e-15 𝐞₂₄ + 1.78e-15 𝐞₃₄,
 52.2]

In [63]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([-5.45779756, -4.11479977,  3.5752481 ,  0.65006836]),
 array([-5.45779756, -4.11479977,  0.65006836,  3.5752481 ]))

In [64]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.309 𝐞₁ + -0.72 𝐞₂ + 0.232 𝐞₃ + 0.576 𝐞₄,
  -0.406 𝐞₁ + -0.214 𝐞₂ + 0.806 𝐞₃ + -0.375 𝐞₄,
  -0.743 𝐞₁ + 0.187 𝐞₂ + -0.0257 𝐞₃ + 0.642 𝐞₄,
  -0.434 𝐞₁ + -0.633 𝐞₂ + -0.545 𝐞₃ + -0.339 𝐞₄],
 [-0.309 𝐞₁ + 0.72 𝐞₂ + -0.232 𝐞₃ + -0.576 𝐞₄,
  0.406 𝐞₁ + 0.214 𝐞₂ + -0.806 𝐞₃ + 0.375 𝐞₄,
  0.743 𝐞₁ + -0.187 𝐞₂ + 0.0257 𝐞₃ + -0.642 𝐞₄,
  -0.434 𝐞₁ + -0.633 𝐞₂ + -0.545 𝐞₃ + -0.339 𝐞₄])

In [65]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-5.45779756 -5.45779756 -5.45779756 -5.45779756]
[29.78755426 29.78755426 29.78755426 29.78755426 29.78755426 29.78755426]


In [66]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([-0.743 𝐞₁ + 0.187 𝐞₂ + -0.0257 𝐞₃ + 0.642 𝐞₄,
  0.358 𝐞₁₂ + -0.343 𝐞₁₃ + -0.118 𝐞₁₄ + 0.531 𝐞₂₃ + -0.393 𝐞₂₄ + 0.551 𝐞₃₄,
  -0.434 𝐞₁ + -0.633 𝐞₂ + -0.545 𝐞₃ + -0.339 𝐞₄],
 [-0.358 𝐞₁₂ + 0.343 𝐞₁₃ + 0.118 𝐞₁₄ + -0.531 𝐞₂₃ + 0.393 𝐞₂₄ + -0.551 𝐞₃₄,
  0.743 𝐞₁ + -0.187 𝐞₂ + 0.0257 𝐞₃ + -0.642 𝐞₄,
  -0.434 𝐞₁ + -0.633 𝐞₂ + -0.545 𝐞₃ + -0.339 𝐞₄])

In [67]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.9180656 ,  0.14374694, -0.33678807,  0.15187553],
        [ 0.14374694, -0.74780823, -0.590866  ,  0.26645268],
        [-0.33678807, -0.590866  ,  0.3843538 , -0.62427824],
        [ 0.15187553,  0.26645268, -0.62427824, -0.71847997]]),
 [0.9999999999999994, -8.326672684688674e-17, 0.0, 0.9999999999999996],
 array([-1.,  1., -1., -1.]))

In [68]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(-0.676 𝐞₁ + -0.676 𝐞₂ + -0.447 𝐞₃ + -0.355 𝐞₄,
 -0.676 𝐞₁ + -0.676 𝐞₂ + -0.447 𝐞₃ + -0.355 𝐞₄)

In [69]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([0.584 𝐞₁ + 0.584 𝐞₂ + 0.564 𝐞₃,
  1.0 𝐞₄,
  0.479 𝐞₁₂ + -0.512 𝐞₁₃ + 0.582 𝐞₂₃,
  1 𝐞₁₂₃],
 [0.007318501827574986, 0.0, 0.533981488584967, 0.0])

In [70]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(0.275, 0.275)

In [71]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(-0.15 𝐞₁ + 0.45 𝐞₂ + 0.542 𝐞₃ + -2.12 𝐞₄,
 -0.15 𝐞₁ + 0.45 𝐞₂ + 0.542 𝐞₃ + -2.12 𝐞₄,
 )

In [72]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(5.09e-10 + -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄,
 -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄,
 -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄,
 -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄)

In [73]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [74]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  -0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
  2.63 𝐞₁₂₃₄],
 [5.55e-17 + -1.31 𝐞₁₂ + 1.9 𝐞₁₃ + -0.902 𝐞₁₄ + -1.62 𝐞₂₃ + -2.45 𝐞₂₄ + -3.36 𝐞₃₄,
  6.51 + -4.44e-16 𝐞₁₂ + -5.55e-16 𝐞₁₃ + 8.88e-16 𝐞₁₄ + -1.33e-15 𝐞₂₃ + -6.66e-16 𝐞₂₄ + -6.66e-16 𝐞₃₄ + 10.5 𝐞₁₂₃₄,
  9.44e-16 + -8.83 𝐞₁₂ + 6.42 𝐞₁₃ + -4.26 𝐞₁₄ + -2.37 𝐞₂₃ + -4.98 𝐞₂₄ + -3.44 𝐞₃₄,
  6.9])

In [75]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[-0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
 4.42 𝐞₁₂ + -3.21 𝐞₁₃ + 2.13 𝐞₁₄ + 1.18 𝐞₂₃ + 2.49 𝐞₂₄ + 1.72 𝐞₃₄]

In [76]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [77]:
x|F, fm(x), x|F|F, fm(fm(x))

(-0.15 𝐞₁ + 0.45 𝐞₂ + 0.542 𝐞₃ + -2.12 𝐞₄,
 -0.15 𝐞₁ + 0.45 𝐞₂ + 0.542 𝐞₃ + -2.12 𝐞₄,
 -1.18 𝐞₁ + -2.06 𝐞₂ + -4.08 𝐞₃ + -1.39 𝐞₄,
 -1.18 𝐞₁ + -2.06 𝐞₂ + -4.08 𝐞₃ + -1.39 𝐞₄)

In [78]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 6.511362965787674, 6.901238279568452],
 [1,
  -5.551115123125783e-17,
  6.511362965787677,
  -9.43689570931383e-16,
  6.901238279568455],
 array([-5.17875809, -1.33260488]),
 array([2.27568849, 1.15438506]),
 array([-8.53074112e-17+2.27568849j, -8.53074112e-17-2.27568849j,
         1.13062987e-16+1.15438506j,  1.13062987e-16-1.15438506j]),
 [1,
  13.022725931575351,
  56.200324231368185,
  89.87293470331656,
  47.62708979138092])

In [79]:
Ck[1], Ck[1] | Ck[2]

(-0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
 4.42 𝐞₁₂ + -3.21 𝐞₁₃ + 2.13 𝐞₁₄ + 1.18 𝐞₂₃ + 2.49 𝐞₂₄ + 1.72 𝐞₃₄)

In [80]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(-0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
 -0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
 1.05e-15 + -2.6e-17 𝐞₁₂ + -9.11e-17 𝐞₁₃ + 3.47e-17 𝐞₁₄ + 5.55e-17 𝐞₂₄ + -5.55e-17 𝐞₃₄ + 2.63 𝐞₁₂₃₄,
 1.05e-15 + 1.11e-16 𝐞₁₃ + -5.55e-17 𝐞₂₄ + 5.9e-17 𝐞₃₄ + 2.63 𝐞₁₂₃₄,
 -6.51 + 5.25 𝐞₁₂₃₄,
 -1.33 + -6.94e-18 𝐞₁₃ + 2.78e-17 𝐞₁₄ + -4.3e-16 𝐞₁₂₃₄,
 -5.18 + -2.78e-17 𝐞₁₂ + -1.39e-17 𝐞₁₃ + -2.39e-15 𝐞₁₂₃₄)

In [81]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-5.178758090439507, -1.3326048753481676], array([-5.17875809, -1.33260488]))

In [82]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(-0.921 𝐞₁₂ + 0.507 𝐞₁₃ + -0.398 𝐞₁₄ + -0.0269 𝐞₂₃ + -0.224 𝐞₂₄ + 0.135 𝐞₃₄,
 -0.921 𝐞₁₂ + 0.507 𝐞₁₃ + -0.398 𝐞₁₄ + -0.0269 𝐞₂₃ + -0.224 𝐞₂₄ + 0.135 𝐞₃₄,
 0.266 𝐞₁₂ + 0.442 𝐞₁₃ + -0.053 𝐞₁₄ + -0.785 𝐞₂₃ + -0.999 𝐞₂₄ + -1.82 𝐞₃₄,
 0.266 𝐞₁₂ + 0.442 𝐞₁₃ + -0.053 𝐞₁₄ + -0.785 𝐞₂₃ + -0.999 𝐞₂₄ + -1.82 𝐞₃₄)

In [83]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, m+1)]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    linear_eq_matrix = np.array([[np.prod(Lk[i:i+k]) for i in range(m)] for k in range(m)])
    inv_matrix = np.linalg.inv(linear_eq_matrix)
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.dot(inv_matrix, [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [84]:
# From Graded Symmetry Groups: Plane and Simple, 
# instead of solving linear equations, the solutions are given algebraically
def bivector_split_(F, alg: Algebra):  
    # FIXME: it produce commuting bivectors but not blades
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    blades = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        blades.append(even * odd.reverse() / normsq(odd))
    return blades

In [85]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-2.37 + -2.18e-12 𝐞₁ + -2.02e-12 𝐞₂ + 4.03e-13 𝐞₃ + 2.29e-12 𝐞₄ + -2.67e-13 𝐞₁₂ + 3.08e-14 𝐞₁₃ + -1.84e-13 𝐞₁₄ + 2.01e-13 𝐞₂₃ + -1.01e-13 𝐞₂₄ + 2.91e-13 𝐞₃₄ + 6.72e-13 𝐞₁₂₃ + -1.34e-13 𝐞₁₂₄ + -6.12e-13 𝐞₁₃₄ + 6.47e-13 𝐞₂₃₄ + 9.42e-13 𝐞₁₂₃₄,
 [array([-1.27289047e-13,  1.60714632e-13, -1.25722902e-13, -3.13956467e-13,
          2.45600030e-13, -3.10093596e-13]),
  array([1., 1., 1., 1., 1., 1.]),
  array([-1.98674403e-13, -1.98674403e-13, -1.98674403e-13, -1.98674403e-13,
         -1.98674403e-13, -1.98674403e-13]),
  array([ 1.56081303, -1.23619362,  1.58025625,  0.63280876, -0.80893477,
          0.64069173])])

In [86]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

7.11e-14 𝐞₁₂ + 3.55e-14 𝐞₁₃ + 1.42e-14 𝐞₁₄ + -1.42e-14 𝐞₂₄ + -5.68e-14 𝐞₃₄

In [87]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-1.26 𝐞₁ + -0.0756 𝐞₂ + 0.0784 𝐞₃ + 0.275 𝐞₄,
 -1.26 𝐞₁ + -0.0756 𝐞₂ + 0.0784 𝐞₃ + 0.275 𝐞₄ + 8.91e-17 𝐞₁₂₃ + -2.69e-17 𝐞₁₂₄ + -2.13e-17 𝐞₁₃₄ + 4.08e-16 𝐞₂₃₄)

In [88]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(-0.921 𝐞₁₂ + 0.507 𝐞₁₃ + -0.398 𝐞₁₄ + -0.0269 𝐞₂₃ + -0.224 𝐞₂₄ + 0.135 𝐞₃₄,
 -0.921 𝐞₁₂ + 0.507 𝐞₁₃ + -0.398 𝐞₁₄ + -0.0269 𝐞₂₃ + -0.224 𝐞₂₄ + 0.135 𝐞₃₄)

In [89]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [90]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [91]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(-1.16 𝐞₁₂ + -1.07 𝐞₁₃ + 0.815 𝐞₁₄ + -0.395 𝐞₂₃ + 5.26 𝐞₂₄ + 1.37 𝐞₃₄,
 -1.16 𝐞₁₂ + -1.07 𝐞₁₃ + 0.815 𝐞₁₄ + -0.395 𝐞₂₃ + 5.26 𝐞₂₄ + 1.37 𝐞₃₄)

In [92]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(-1.16 𝐞₁₂ + -1.07 𝐞₁₃ + 0.815 𝐞₁₄ + -0.395 𝐞₂₃ + 5.26 𝐞₂₄ + 1.37 𝐞₃₄,
 -1.16 𝐞₁₂ + -1.07 𝐞₁₃ + 0.815 𝐞₁₄ + -0.395 𝐞₂₃ + 5.26 𝐞₂₄ + 1.37 𝐞₃₄)

In [93]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(-1.23 𝐞₁₂ + 0.675 𝐞₁₃ + -0.53 𝐞₁₄ + -0.0358 𝐞₂₃ + -0.299 𝐞₂₄ + 0.18 𝐞₃₄,
 -1.23 𝐞₁₂ + 0.675 𝐞₁₃ + -0.53 𝐞₁₄ + -0.0358 𝐞₂₃ + -0.299 𝐞₂₄ + 0.18 𝐞₃₄)

In [94]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(4.72 𝐞₁ + -5.33 𝐞₂ + -2.49 𝐞₃ + 5.72 𝐞₄,
 0.654 𝐞₁ + 1.63 𝐞₂ + 1.28 𝐞₃ + -7.22 𝐞₄,
 11.5 𝐞₁ + 1.96 𝐞₂ + 9.44 𝐞₃ + 19.8 𝐞₄,
 3.39 𝐞₁ + 15.9 𝐞₂ + 17.0 𝐞₃ + -6.03 𝐞₄)

In [95]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(0.223 𝐞₁ + 1.14 𝐞₂ + 1.14 𝐞₃ + -0.0143 𝐞₄,
 0.223 𝐞₁ + 1.14 𝐞₂ + 1.14 𝐞₃ + -0.0143 𝐞₄)

In [96]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(0.656 𝐞₁ + 0.923 𝐞₂ + -0.527 𝐞₃ + 0.0675 𝐞₄,
 1.44 𝐞₁ + 3.07 𝐞₂ + -1.35 𝐞₃ + 0.393 𝐞₄ + 6.69e-16 𝐞₁₂₃ + 1.04e-16 𝐞₁₂₄ + -1.05e-16 𝐞₁₃₄ + 1.59e-16 𝐞₂₃₄)

In [97]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(-0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
 -0.655 𝐞₁₂ + 0.949 𝐞₁₃ + -0.451 𝐞₁₄ + -0.811 𝐞₂₃ + -1.22 𝐞₂₄ + -1.68 𝐞₃₄,
 -1.06 𝐞₁₂ + 0.585 𝐞₁₃ + -0.459 𝐞₁₄ + -0.031 𝐞₂₃ + -0.259 𝐞₂₄ + 0.156 𝐞₃₄,
 -1.06 𝐞₁₂ + 0.585 𝐞₁₃ + -0.459 𝐞₁₄ + -0.031 𝐞₂₃ + -0.259 𝐞₂₄ + 0.156 𝐞₃₄)

In [98]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([ -0.37815445,  -0.48640434,   0.73578706, 253.81595428,
        -2.2491517 , -45.14961975])

In [99]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.25101574, 0.25101574, 0.25101574, 0.25101574, 0.25101574,
        0.25101574])]

In [100]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([0.5010147051239883], array([0.25101573]), [0.2510157365077086])

In [101]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.473 𝐞₁ + 0.0261 𝐞₂ + -0.977 𝐞₃ + -0.175 𝐞₄],
 [-0.473 𝐞₁ + 0.0261 𝐞₂ + -0.977 𝐞₃ + -0.175 𝐞₄ + -6.94e-18 𝐞₁₂₄ + -6.94e-18 𝐞₁₃₄ + -1.39e-17 𝐞₂₃₄],
 [-0.473 𝐞₁ + 0.0261 𝐞₂ + -0.977 𝐞₃ + -0.175 𝐞₄ + 1.04e-17 𝐞₁₂₄ + 2.78e-17 𝐞₂₃₄])

In [102]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.676 𝐞₁ + 0.738 𝐞₂ + 0.369 𝐞₃ + 0.269 𝐞₄],
 [0.676 𝐞₁ + 0.738 𝐞₂ + 0.369 𝐞₃ + 0.269 𝐞₄ + -1.04e-17 𝐞₁₂₄ + -2.78e-17 𝐞₂₃₄],
 [0.676 𝐞₁ + 0.738 𝐞₂ + 0.369 𝐞₃ + 0.269 𝐞₄ + 6.94e-18 𝐞₁₂₄ + 6.94e-18 𝐞₁₃₄ + 1.39e-17 𝐞₂₃₄])

In [103]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.0999999994011997], [1.0999999994011997])

In [104]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([0.501 + 0.366 𝐞₁₂ + -0.485 𝐞₁₃ + 0.00889 𝐞₁₄ + -0.729 𝐞₂₃ + -0.136 𝐞₂₄ + 0.198 𝐞₃₄],
 [0.501 + 0.366 𝐞₁₂ + -0.485 𝐞₁₃ + 0.00889 𝐞₁₄ + -0.729 𝐞₂₃ + -0.136 𝐞₂₄ + 0.198 𝐞₃₄])

In [105]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.5010147051239884

In [106]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 4.32385277, -3.22385277])

In [107]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(0.656 𝐞₁ + 0.923 𝐞₂ + -0.527 𝐞₃ + 0.0675 𝐞₄,
 0.656 𝐞₁ + 0.923 𝐞₂ + -0.527 𝐞₃ + 0.0675 𝐞₄ + -1.39e-17 𝐞₁₂₃ + -1.73e-18 𝐞₁₂₄ + -5.2e-18 𝐞₂₃₄)

In [108]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-3.22385277,  0.50101471,  0.50101471,  4.32385277]),
 [-0.781 𝐞₁ + 0.489 𝐞₂ + 0.372 𝐞₃ + 0.111 𝐞₄,
  0.618 𝐞₁ + 0.642 𝐞₂ + 0.381 𝐞₃ + 0.245 𝐞₄,
  -0.054 𝐞₁ + 0.549 𝐞₂ + -0.834 𝐞₃ + -0.00671 𝐞₄,
  -0.0674 𝐞₁ + -0.216 𝐞₂ + -0.146 𝐞₃ + 0.963 𝐞₄])

In [109]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([10.39322669, 10.39322669, 10.39322669, 10.39322669]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([18.69570278, 18.69570278, 18.69570278, 18.69570278])]

In [110]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(0.656 𝐞₁ + 0.923 𝐞₂ + -0.527 𝐞₃ + 0.0675 𝐞₄,
 0.656 𝐞₁ + 0.923 𝐞₂ + -0.527 𝐞₃ + 0.0675 𝐞₄ + 2.36e-17 𝐞₁₂₃ + 1.23e-17 𝐞₁₂₄ + -3.66e-19 𝐞₁₃₄ + -2.34e-17 𝐞₂₃₄)

In [111]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.0888 + 0.544 𝐞₁ + 0.546 𝐞₂ + 0.392 𝐞₃ + 0.942 𝐞₄ + 0.865 𝐞₁₂ + 0.511 𝐞₁₃ + 0.346 𝐞₁₄ + 0.01 𝐞₂₃ + 0.615 𝐞₂₄ + 0.305 𝐞₃₄ + 0.499 𝐞₁₂₃ + 0.775 𝐞₁₂₄ + 0.812 𝐞₁₃₄ + 0.374 𝐞₂₃₄ + 0.133 𝐞₁₂₃₄,
 0.0888 + -0.824 𝐞₁ + -1.56 𝐞₂ + -0.984 𝐞₃ + 12.9 𝐞₄ + 25.1 𝐞₁₂ + 16.5 𝐞₁₃ + -46.9 𝐞₁₄ + 0.879 𝐞₂₃ + 49.1 𝐞₂₄ + 33.4 𝐞₃₄ + 12.9 𝐞₁₂₃ + 2.03e+02 𝐞₁₂₄ + 1.66e+02 𝐞₁₃₄ + 7.22 𝐞₂₃₄ + 37.7 𝐞₁₂₃₄)

In [112]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.0888 + 0.544 𝐞₁ + 0.546 𝐞₂ + 0.392 𝐞₃ + 0.942 𝐞₄ + 0.865 𝐞₁₂ + 0.511 𝐞₁₃ + 0.346 𝐞₁₄ + 0.01 𝐞₂₃ + 0.615 𝐞₂₄ + 0.305 𝐞₃₄ + 0.499 𝐞₁₂₃ + 0.775 𝐞₁₂₄ + 0.812 𝐞₁₃₄ + 0.374 𝐞₂₃₄ + 0.133 𝐞₁₂₃₄,
 0.0888 + 0.544 𝐞₁ + 0.546 𝐞₂ + 0.392 𝐞₃ + 0.942 𝐞₄ + 0.865 𝐞₁₂ + 0.511 𝐞₁₃ + 0.346 𝐞₁₄ + 0.01 𝐞₂₃ + 0.615 𝐞₂₄ + 0.305 𝐞₃₄ + 0.499 𝐞₁₂₃ + 0.775 𝐞₁₂₄ + 0.812 𝐞₁₃₄ + 0.374 𝐞₂₃₄ + 0.133 𝐞₁₂₃₄)

In [113]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄ + -5.73e-17 𝐞₁₂₃ + 2.54e-17 𝐞₁₂₄ + -1.33e-17 𝐞₁₃₄ + -4.76e-17 𝐞₂₃₄,
 0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄ + 1.18e-16 𝐞₁₂₃ + -7.49e-18 𝐞₁₂₄ + 1.15e-18 𝐞₁₃₄ + 3.39e-17 𝐞₂₃₄)

In [114]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([4.41e-16 + -0.804 𝐞₁₂ + -0.527 𝐞₁₃ + -0.26 𝐞₁₄ + -0.0524 𝐞₂₃ + 0.0484 𝐞₂₄ + 0.0487 𝐞₃₄,
  8.65e-17 + 0.374 𝐞₁₂ + -0.495 𝐞₁₃ + 0.00908 𝐞₁₄ + -0.745 𝐞₂₃ + -0.139 𝐞₂₄ + 0.202 𝐞₃₄,
  -1.95e-16 + 0.0487 𝐞₁₂ + -0.0484 𝐞₁₃ + -0.0524 𝐞₁₄ + -0.26 𝐞₂₃ + 0.527 𝐞₂₄ + -0.804 𝐞₃₄],
 [0.0406 𝐞₁₂ + 0.211 𝐞₁₃ + 0.802 𝐞₁₄ + -0.0974 𝐞₂₃ + -0.435 𝐞₂₄ + -0.334 𝐞₃₄,
  0.374 𝐞₁₂ + -0.495 𝐞₁₃ + 0.00908 𝐞₁₄ + -0.745 𝐞₂₃ + -0.139 𝐞₂₄ + 0.202 𝐞₃₄,
  0.374 𝐞₁₂ + -0.495 𝐞₁₃ + 0.00908 𝐞₁₄ + -0.745 𝐞₂₃ + -0.139 𝐞₂₄ + 0.202 𝐞₃₄,
  0.0183 𝐞₁₂ + 0.087 𝐞₁₃ + -0.142 𝐞₁₄ + 0.236 𝐞₂₃ + -0.187 𝐞₂₄ + 0.939 𝐞₃₄])

In [115]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([-0.781 𝐞₁ + 0.489 𝐞₂ + 0.372 𝐞₃ + 0.111 𝐞₄,
  0.618 𝐞₁ + 0.642 𝐞₂ + 0.381 𝐞₃ + 0.245 𝐞₄,
  -0.054 𝐞₁ + 0.549 𝐞₂ + -0.834 𝐞₃ + -0.00671 𝐞₄,
  -0.0674 𝐞₁ + -0.216 𝐞₂ + -0.146 𝐞₃ + 0.963 𝐞₄],
 [[-0.781 𝐞₁ + 0.489 𝐞₂ + 0.372 𝐞₃ + 0.111 𝐞₄,
   -0.781 𝐞₁ + 0.489 𝐞₂ + 0.372 𝐞₃ + 0.111 𝐞₄,
   -0.781 𝐞₁ + 0.489 𝐞₂ + 0.372 𝐞₃ + 0.111 𝐞₄],
  [0.234 𝐞₁ + 0.781 𝐞₂ + -0.569 𝐞₃ + 0.106 𝐞₄,
   0.234 𝐞₁ + 0.781 𝐞₂ + -0.569 𝐞₃ + 0.106 𝐞₄,
   0.618 𝐞₁ + 0.642 𝐞₂ + 0.381 𝐞₃ + 0.245 𝐞₄],
  [-0.575 𝐞₁ + -0.322 𝐞₂ + -0.719 𝐞₃ + -0.221 𝐞₄,
   -0.575 𝐞₁ + -0.322 𝐞₂ + -0.719 𝐞₃ + -0.221 𝐞₄,
   -0.054 𝐞₁ + 0.549 𝐞₂ + -0.834 𝐞₃ + -0.00671 𝐞₄],
  [-0.0674 𝐞₁ + -0.216 𝐞₂ + -0.146 𝐞₃ + 0.963 𝐞₄,
   -0.0674 𝐞₁ + -0.216 𝐞₂ + -0.146 𝐞₃ + 0.963 𝐞₄,
   -0.0674 𝐞₁ + -0.216 𝐞₂ + -0.146 𝐞₃ + 0.963 𝐞₄]])

In [116]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄ + -5.73e-17 𝐞₁₂₃ + 2.54e-17 𝐞₁₂₄ + -1.33e-17 𝐞₁₃₄ + -4.76e-17 𝐞₂₃₄,
 0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄)

In [117]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(-0.262 + 0.509 𝐞₁₂ + -0.2 𝐞₁₃ + 0.197 𝐞₁₄ + -0.0924 𝐞₂₃ + -0.0201 𝐞₂₄ + -0.315 𝐞₃₄ + 0.697 𝐞₁₂₃₄,
 -0.262 + 0.509 𝐞₁₂ + -0.2 𝐞₁₃ + 0.197 𝐞₁₄ + -0.0924 𝐞₂₃ + -0.0201 𝐞₂₄ + -0.315 𝐞₃₄ + 0.697 𝐞₁₂₃₄)

In [118]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.404 + -0.73 𝐞₁₂ + 0.401 𝐞₁₃ + -0.315 𝐞₁₄ + -0.0213 𝐞₂₃ + -0.178 𝐞₂₄ + 0.107 𝐞₃₄,
 0.404 + -0.73 𝐞₁₂ + 0.401 𝐞₁₃ + -0.315 𝐞₁₄ + -0.0213 𝐞₂₃ + -0.178 𝐞₂₄ + 0.107 𝐞₃₄)

In [119]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(0.741 𝐞₁ + 0.502 𝐞₂ + 0.619 𝐞₃ + 0.234 𝐞₄,
 0.741 𝐞₁ + 0.502 𝐞₂ + 0.619 𝐞₃ + 0.234 𝐞₄)

In [120]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄ + -5.73e-17 𝐞₁₂₃ + 2.54e-17 𝐞₁₂₄ + -1.33e-17 𝐞₁₃₄ + -4.76e-17 𝐞₂₃₄,
 0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄,
 0.475 𝐞₁ + 0.897 𝐞₂ + -0.433 𝐞₃ + 0.145 𝐞₄)

In [121]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(-0.225 𝐞₁₂₃ + -0.666 𝐞₁₂₄ + 0.143 𝐞₁₃₄ + -0.696 𝐞₂₃₄,
 -0.225 𝐞₁₂₃ + -0.666 𝐞₁₂₄ + 0.143 𝐞₁₃₄ + -0.696 𝐞₂₃₄)

In [122]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(0.862 𝐞₁₂ + -0.435 𝐞₁₃ + -0.592 𝐞₁₄ + -0.187 𝐞₂₃ + 1.7 𝐞₂₄ + -0.985 𝐞₃₄,
 0.862 𝐞₁₂ + -0.435 𝐞₁₃ + -0.592 𝐞₁₄ + -0.187 𝐞₂₃ + 1.7 𝐞₂₄ + -0.985 𝐞₃₄)

In [123]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(-0.79 𝐞₁ + 1.63 𝐞₂ + 0.393 𝐞₃ + -0.249 𝐞₄ + -1.2 𝐞₁₂₃ + -0.0312 𝐞₁₂₄ + -0.359 𝐞₁₃₄ + 0.345 𝐞₂₃₄,
 -0.79 𝐞₁ + 1.63 𝐞₂ + 0.393 𝐞₃ + -0.249 𝐞₄ + -1.2 𝐞₁₂₃ + -0.0312 𝐞₁₂₄ + -0.359 𝐞₁₃₄ + 0.345 𝐞₂₃₄)

In [124]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = create_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-3.41 + 5.55e-17 𝐞₁₂ + -1.39e-17 𝐞₁₅ + -4.16e-17 𝐞₂₅ + 0.886 𝐞₁₂₃₄ + -0.764 𝐞₁₂₃₅ + -0.749 𝐞₁₂₄₅ + 0.3 𝐞₁₃₄₅ + 0.309 𝐞₂₃₄₅

In [125]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.797 𝐞₁ + 0.527 𝐞₂ + 0.56 𝐞₃ + 0.62 𝐞₄ + 0.314 𝐞₅,
 0.797 𝐞₁ + 0.527 𝐞₂ + 0.56 𝐞₃ + 0.62 𝐞₄ + 0.314 𝐞₅)

In [126]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part
# with eigenvalue 1 or -1: identity or a reflection

skew_blades = bivector_split(ortho_curl/2, alg)
sym_vec = reduce(alg.ip, skew_blades, alg.pseudoscalar((1,)))
sym_vec = sym_vec / norm(sym_vec)
p = 0
if max_diff(ortho(sym_vec), -sym_vec) < 1e-6:
    p += 1

In [127]:
rotors = []
for blade in skew_blades:
    t = P(sum(alg.frame), blade)
    # blade_norm = norm(blade)
    # we can't simply rely on ortho_m:
    # angle = np.arcsin(blade_norm) has no unique solution in [0, pi].
    # (pi - angle) generates the same sine, but differnt cosine (alpha).
    
    alpha = np.median(terms_ratio(ortho(t) - (t | ortho_curl)/2, t))
    # half_blade_ortho = (angle/blade_norm/2) * blade.reverse()
    # rotors.append(blade_exp(half_blade_ortho))
    rotors.append(simple_rotor_sqrt(alpha + blade))
rotor = reduce(alg.gp, rotors)
if p == 1:
    rotor2 = sym_vec * rotor
else:
    rotor2 = rotor
(-1) ** p * rotor2.reverse().sw(x), ortho(x)

(-0.032 𝐞₁ + -0.265 𝐞₂ + 0.738 𝐞₃ + 0.825 𝐞₄ + -0.642 𝐞₅ + -6.94e-17 𝐞₁₂₃₄₅,
 -0.032 𝐞₁ + -0.265 𝐞₂ + 0.738 𝐞₃ + 0.825 𝐞₄ + -0.642 𝐞₅)

In [128]:
# On the other hand, we can solve the rotor part by Lasenby's method
frame = [item for blade in skew_blades for item in blade_split(blade, alg)]
r_frame = reciprocal(frame)
lasenby_rotor = sum(char_multi_linear(ortho, s, alg, frame, r_frame) for s in range(5))
terms_ratio(rotor, lasenby_rotor)

array([0.11816824, 0.11816824, 0.11816824, 0.11816824, 0.11816824,
       0.11816824, 0.11816824, 0.11816824, 0.11816824, 0.11816824,
       0.11816824, 0.11816824, 0.11816824, 0.11816824, 0.11816824,
       0.11816824])

In [129]:
# For orthogonal transform, we simply normalize it
l_rotor = lasenby_rotor/norm(lasenby_rotor)
if p == 1:
    l_rotor = l_rotor * sym_vec
(-1) ** p * l_rotor.reverse().sw(x), ortho(x)

(-0.032 𝐞₁ + -0.265 𝐞₂ + 0.738 𝐞₃ + 0.825 𝐞₄ + -0.642 𝐞₅ + 6.94e-17 𝐞₁₂₃₄₅,
 -0.032 𝐞₁ + -0.265 𝐞₂ + 0.738 𝐞₃ + 0.825 𝐞₄ + -0.642 𝐞₅)